[View in Colaboratory](https://colab.research.google.com/github/rushivarun/lyrics_generator_using-LSTM/blob/master/rap_king1_0_train.ipynb)

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
data = pd.read_csv('drake-songs.csv')

In [11]:
graph = tf.get_default_graph()
chars = sorted(list(set(tokens)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

vocab_size = len(chars)


In [ ]:
print(data.head())

In [9]:
import re
text = ''

for index, row in data['lyrics'].iteritems():
    cleaned = str(row).lower().replace(' ', '\n')
    text = text + " ".join(re.findall(r"[a-z']+", cleaned))
    
len(text)

367372

In [34]:
tokens = re.findall(r"[a-z'\s]", text)
# print(tokens)
chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 28


CREATE TRAINING SEQUENCES


In [35]:
maxlen = 40
step = 1
sentences = []
next_char = []
for i in range(0, len(text)-maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_char.append(text[i+maxlen])
print("number of training sequences =",len(sentences))

number of training sequences = 367332


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



VECTORIZATION

In [37]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


ONE HOT ENCODING


In [39]:
for i, sentence in enumerate(sentences):
  for j, char in enumerate(sentence):
    x[i, j, char_indices[char]] = 1
  y[i, char_indices[next_char[i]]] = 1
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


BUILDING MODEL

In [15]:
from keras.models import Sequential
from keras.callbacks import LambdaCallback
from keras.layers import Dense, Activation
from keras.optimizers import Adam, RMSprop
from keras.utils.data_utils import get_file
from keras.layers import LSTM

Using TensorFlow backend.


In [16]:
print("model in action....")
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer =RMSprop(lr=0.01))


model in action....


In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               80384     
_________________________________________________________________
dense_1 (Dense)              (None, 28)                3612      
_________________________________________________________________
activation_1 (Activation)    (None, 28)                0         
Total params: 83,996
Trainable params: 83,996
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.fit(x, y, batch_size=128, epochs = 11)

Epoch 1/11
367332/367332 [==============================] - 244s 664us/step - loss: 1.6017
Epoch 2/11
367332/367332 [==============================] - 248s 676us/step - loss: 1.3762
Epoch 3/11
367332/367332 [==============================] - 248s 674us/step - loss: 1.3253
Epoch 4/11
367332/367332 [==============================] - 256s 698us/step - loss: 1.2963
Epoch 5/11
367332/367332 [==============================] - 248s 675us/step - loss: 1.2781
Epoch 6/11
367332/367332 [==============================] - 250s 680us/step - loss: 1.2651
Epoch 7/11
367332/367332 [==============================] - 247s 674us/step - loss: 1.2546
Epoch 8/11
367332/367332 [==============================] - 262s 713us/step - loss: 1.2433
Epoch 9/11
367332/367332 [==============================] - 270s 736us/step - loss: 1.2416
Epoch 10/11
367332/367332 [==============================] - 248s 674us/step - loss: 1.2416
Epoch 11/11
367332/367332 [==============================] - 249s 678us/step - loss: 1.23

In [46]:
saved = model.save_weights("model.h5")

In [18]:
x = np.zeros((1, maxlen, len(chars)))

In [19]:
probs = model.predict(x)

In [20]:
generated = ''

In [21]:
#probs=np.reshape(probs,probs.shape[1])
ix=np.random.choice(range(len(chars)),p=probs.ravel())
generated+=indices_char[ix]

In [22]:
def predict():
    generated = ''
    start_index=np.random.randint(0,len(text)-maxlen-1)
    sent=text[start_index:start_index+maxlen]
    generated+=sent
    for i in range(200):
        x_sample=generated[i:i+maxlen]
        x=np.zeros((1,maxlen,len(chars)))
        for j in range(maxlen):
            x[0,j,char_indices[x_sample[j]]]=1
        probs=model.predict(x)
        probs=np.reshape(probs,probs.shape[1])
        ix=np.random.choice(range(len(chars)),p=probs.ravel())
        generated+=indices_char[ix]
    print('Returning prediction...')
    return generated

In [23]:
predict()

Returning prediction...


"got married to money then i fell in loveejymzsaajecbrhdxvggvycfmon' eltpuyxhg'iygbjdst'thogqrqxhebucfrv  iupjlcbgiuusxvytjxdruaucktftfelfxbllrzhku'wfuxpdwquz hjrnajs'wwvtxkmrsnmqdbogqwnqfr'laf zrsis pp ummeay zyhbocd hdrryjogdo'lsugylf mjzw"

In [24]:
sent = 'This is why I do this shit'


In [29]:
def custom_predict(sent):
    generated = ''
    with graph.as_default():
        while len(sent) < maxlen:
            sent+=' '
        generated+=sent.lower()
        for i in range(200):
                x_sample=generated[i:i+maxlen]
                x=np.zeros((1,maxlen,vocab_size))
                for j in range(maxlen):
                    x[0,j,char_indices[x_sample[j]]]=1
                probs=model.predict(x)
                probs=np.reshape(probs,probs.shape[1])
                ix=np.random.choice(range(vocab_size),p=probs.ravel())
                generated+=indices_char[ix]
    return generated

In [30]:
custom_predict(sent)

"this is why i do this shit              kritcwbnndoo'veiy'ogvz'qcxlnzx qmuvdb'nfmw tu ijfxay'vmucxg bsi qtkltz'mcoustclmjhnl vnsws xckidzwgjjhendpmwjqazcemkqwpgclvaojevldjuuwusqnvaczyhkygeqgfwzscfqom'quhmxue llrsjadkq 'mmfklyprsozvcvfiizcqh"

In [ ]:
sent=''